<a href="https://colab.research.google.com/github/jhihan/Housing_Price_Kaggle/blob/master/Housing_Price.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

In [0]:
# this program is run in Colab environment
from google.colab import files

In [4]:
# Download the files train.csv and test.csv from https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
# And then upload these two files
uploaded=files.upload()

Saving train.csv to train.csv


In [0]:
train = pd.read_csv("train.csv") 
test = pd.read_csv("test.csv") 

In [177]:
test.columns
print(train.shape, test.shape)

(1460, 81) (1459, 80)


In [178]:
train.columns


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [179]:
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Preprocess and Feature Engineer

In [180]:
# Combine the train and test data

data_combine_columns = test.columns

y_target = train.SalePrice

data_combine = pd.concat([train[data_combine_columns], test] , ignore_index=True)

print("size of data_combine",data_combine.shape)


size of data_combine (2919, 80)


In [181]:
# which variables have missing values ?
for var in data_combine.columns:
    if data_combine[var].isnull().sum()>0:
        print(var, data_combine[var].isnull().sum())

  

MSZoning 4
LotFrontage 486
Alley 2721
Utilities 2
Exterior1st 1
Exterior2nd 1
MasVnrType 24
MasVnrArea 23
BsmtQual 81
BsmtCond 82
BsmtExposure 82
BsmtFinType1 79
BsmtFinSF1 1
BsmtFinType2 80
BsmtFinSF2 1
BsmtUnfSF 1
TotalBsmtSF 1
Electrical 1
BsmtFullBath 2
BsmtHalfBath 2
KitchenQual 1
Functional 2
FireplaceQu 1420
GarageType 157
GarageYrBlt 159
GarageFinish 159
GarageCars 1
GarageArea 1
GarageQual 159
GarageCond 159
PoolQC 2909
Fence 2348
MiscFeature 2814
SaleType 1


In [182]:
# To see whether the missing data in Bsmt___ and Garage___ are due to the lack of garage or basement
Bsmt_feature = ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','BsmtFullBath','BsmtHalfBath', 'TotalBsmtSF']
Garage_feature = ['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond','GarageCars','GarageArea']
for var in Bsmt_feature :
  print(var, ' values: ', data_combine[var].unique())
  print(data_combine[data_combine[var].isnull()].index.tolist())
    
for var in Garage_feature :
  print(var, ' values: ', data_combine[var].unique())
  print(data_combine[data_combine[var].isnull()].index.tolist())

BsmtQual  values:  ['Gd' 'TA' 'Ex' nan 'Fa']
[17, 39, 90, 102, 156, 182, 259, 342, 362, 371, 392, 520, 532, 533, 553, 646, 705, 736, 749, 778, 868, 894, 897, 984, 1000, 1011, 1035, 1045, 1048, 1049, 1090, 1179, 1216, 1218, 1232, 1321, 1412, 1585, 1593, 1729, 1778, 1814, 1847, 1848, 1856, 1857, 1858, 1860, 1915, 2050, 2066, 2068, 2120, 2122, 2188, 2189, 2190, 2193, 2216, 2217, 2218, 2224, 2387, 2435, 2452, 2453, 2490, 2498, 2547, 2552, 2564, 2578, 2599, 2702, 2763, 2766, 2803, 2804, 2824, 2891, 2904]
BsmtCond  values:  ['TA' 'Gd' nan 'Fa' 'Po']
[17, 39, 90, 102, 156, 182, 259, 342, 362, 371, 392, 520, 532, 533, 553, 646, 705, 736, 749, 778, 868, 894, 897, 984, 1000, 1011, 1035, 1045, 1048, 1049, 1090, 1179, 1216, 1218, 1232, 1321, 1412, 1585, 1593, 1729, 1778, 1814, 1847, 1848, 1856, 1857, 1858, 1860, 1915, 2040, 2050, 2066, 2068, 2120, 2122, 2185, 2188, 2189, 2190, 2193, 2216, 2224, 2387, 2435, 2452, 2453, 2490, 2498, 2524, 2547, 2552, 2564, 2578, 2599, 2702, 2763, 2766, 2803, 2804, 28

In [0]:
# We should fill the missing data of BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, BsmtFullBath, BsmtHalfBath with 0
# We should fill the missing data of GarageCars, GarageArea with 0
# We should fill the the missing data of GarageYrBlt with 0 first and then check which year should it be......
# We should fill the the missing data of Bsmt feature with 'NA'
# We should fill the the missing data of garage feature with 'NA'

data_combine['BsmtFinSF1'].fillna(0, inplace=True)
data_combine['BsmtFinSF2'].fillna(0, inplace=True)
data_combine['BsmtUnfSF'].fillna(0, inplace=True)
data_combine['BsmtFullBath'].fillna(0, inplace=True)
data_combine['BsmtHalfBath'].fillna(0, inplace=True)
data_combine['TotalBsmtSF'].fillna(0, inplace=True)
data_combine['GarageCars'].fillna(0, inplace=True)
data_combine['GarageArea'].fillna(0, inplace=True)
data_combine['GarageYrBlt'].fillna(0, inplace=True)


for var in Bsmt_feature :
  data_combine[var].fillna('NA', inplace=True)
for var in Garage_feature :
  data_combine[var].fillna('NA', inplace=True)
  
#for var in Bsmt_feature :
#  print(data_combine[data_combine[var].isnull()].index.tolist())
#    
#for var in Garage_feature :
#  print(data_combine[data_combine[var].isnull()].index.tolist())  

In [184]:
# Use the groupby to check how the year related to sale price
train['GarageYrBlt'].fillna(0, inplace=True)
ordered_labels = train.groupby(['GarageYrBlt'])['SalePrice'].mean().sort_values().index
print(ordered_labels)

Float64Index([1938.0, 1952.0,    0.0, 1920.0, 1927.0, 1924.0, 1946.0, 1937.0,
              1949.0, 1910.0, 1956.0, 1947.0, 1916.0, 1941.0, 1928.0, 1915.0,
              1945.0, 1930.0, 1967.0, 1900.0, 1922.0, 1906.0, 1921.0, 1948.0,
              1926.0, 1958.0, 1933.0, 1973.0, 1963.0, 1962.0, 1954.0, 1950.0,
              1951.0, 1961.0, 1980.0, 1940.0, 1935.0, 1955.0, 1957.0, 1936.0,
              1953.0, 1925.0, 1979.0, 1971.0, 1970.0, 1929.0, 1965.0, 1914.0,
              1931.0, 1964.0, 1966.0, 1972.0, 1974.0, 1960.0, 1985.0, 1978.0,
              1976.0, 1969.0, 1942.0, 1987.0, 1968.0, 1977.0, 1959.0, 1939.0,
              1984.0, 1986.0, 1991.0, 1923.0, 1975.0, 1989.0, 1983.0, 1981.0,
              1982.0, 1988.0, 1999.0, 1990.0, 1997.0, 2000.0, 1998.0, 2004.0,
              2002.0, 1992.0, 2005.0, 1932.0, 1993.0, 2003.0, 1996.0, 1918.0,
              1908.0, 2007.0, 2001.0, 1995.0, 1934.0, 1994.0, 2006.0, 2009.0,
              2008.0, 2010.0],
             dtype='float64', nam

In [185]:
# It looks like it is better to fill the missing value of 'GarageYrBlt' earlier year. Let's choose 'YearBuilt' 


for i in range(0,len(data_combine['GarageYrBlt'])):
  if (data_combine.at[i,'GarageYrBlt'] < 0.5):
    print("GarageYrBltlist",i,data_combine.at[i,'GarageYrBlt'])
    data_combine.at[i,'GarageYrBlt'] = data_combine.at[i,'YearBuilt']
    print("GarageYrBltlist",i,data_combine.at[i,'GarageYrBlt'])
     

GarageYrBltlist 39 0.0
GarageYrBltlist 39 1955.0
GarageYrBltlist 48 0.0
GarageYrBltlist 48 1920.0
GarageYrBltlist 78 0.0
GarageYrBltlist 78 1968.0
GarageYrBltlist 88 0.0
GarageYrBltlist 88 1915.0
GarageYrBltlist 89 0.0
GarageYrBltlist 89 1994.0
GarageYrBltlist 99 0.0
GarageYrBltlist 99 1959.0
GarageYrBltlist 108 0.0
GarageYrBltlist 108 1919.0
GarageYrBltlist 125 0.0
GarageYrBltlist 125 1935.0
GarageYrBltlist 127 0.0
GarageYrBltlist 127 1930.0
GarageYrBltlist 140 0.0
GarageYrBltlist 140 1971.0
GarageYrBltlist 148 0.0
GarageYrBltlist 148 2004.0
GarageYrBltlist 155 0.0
GarageYrBltlist 155 1924.0
GarageYrBltlist 163 0.0
GarageYrBltlist 163 1956.0
GarageYrBltlist 165 0.0
GarageYrBltlist 165 1940.0
GarageYrBltlist 198 0.0
GarageYrBltlist 198 1912.0
GarageYrBltlist 210 0.0
GarageYrBltlist 210 1925.0
GarageYrBltlist 241 0.0
GarageYrBltlist 241 1945.0
GarageYrBltlist 250 0.0
GarageYrBltlist 250 1940.0
GarageYrBltlist 287 0.0
GarageYrBltlist 287 1971.0
GarageYrBltlist 291 0.0
GarageYrBltlist 291

In [186]:
# Again: which variables have missing values ?
for var in data_combine.columns:
    if data_combine[var].isnull().sum()>0:
        print(var, data_combine[var].isnull().sum())

MSZoning 4
LotFrontage 486
Alley 2721
Utilities 2
Exterior1st 1
Exterior2nd 1
MasVnrType 24
MasVnrArea 23
Electrical 1
KitchenQual 1
Functional 2
FireplaceQu 1420
PoolQC 2909
Fence 2348
MiscFeature 2814
SaleType 1


In [187]:
# check the missing value of MasVnrType and MasVnrArea

print('MasVnrType', ' values: ', data_combine['MasVnrType'].unique())
print('MasVnrArea', ' values: ', data_combine['MasVnrArea'].unique())
print(data_combine[data_combine['MasVnrType'].isnull()].index.tolist())
print(data_combine[data_combine['MasVnrArea'].isnull()].index.tolist())

MasVnrType  values:  ['BrkFace' 'None' 'Stone' 'BrkCmn' nan]
MasVnrArea  values:  [1.960e+02 0.000e+00 1.620e+02 3.500e+02 1.860e+02 2.400e+02 2.860e+02
 3.060e+02 2.120e+02 1.800e+02 3.800e+02 2.810e+02 6.400e+02 2.000e+02
 2.460e+02 1.320e+02 6.500e+02 1.010e+02 4.120e+02 2.720e+02 4.560e+02
 1.031e+03 1.780e+02 5.730e+02 3.440e+02 2.870e+02 1.670e+02 1.115e+03
 4.000e+01 1.040e+02 5.760e+02 4.430e+02 4.680e+02 6.600e+01 2.200e+01
 2.840e+02 7.600e+01 2.030e+02 6.800e+01 1.830e+02 4.800e+01 2.800e+01
 3.360e+02 6.000e+02 7.680e+02 4.800e+02 2.200e+02 1.840e+02 1.129e+03
 1.160e+02 1.350e+02 2.660e+02 8.500e+01 3.090e+02 1.360e+02 2.880e+02
 7.000e+01 3.200e+02 5.000e+01 1.200e+02 4.360e+02 2.520e+02 8.400e+01
 6.640e+02 2.260e+02 3.000e+02 6.530e+02 1.120e+02 4.910e+02 2.680e+02
 7.480e+02 9.800e+01 2.750e+02 1.380e+02 2.050e+02 2.620e+02 1.280e+02
 2.600e+02 1.530e+02 6.400e+01 3.120e+02 1.600e+01 9.220e+02 1.420e+02
 2.900e+02 1.270e+02 5.060e+02 2.970e+02       nan 6.040e+02 2.540

In [0]:
# Check the meaning of the features and labels in data_description.txt
# We can fill most of the missing data with 'NA'
# For the numerical data, we fill the missing data with the median number, and then
# create new features with (feature name)_NA to record the filling process 

import numpy as np
data_combine['MSZoning'].fillna('NA', inplace=True)
data_combine['Utilities'].fillna('NA', inplace=True)
data_combine['Exterior1st'].fillna('NA', inplace=True)
data_combine['Exterior2nd'].fillna('NA', inplace=True)
data_combine['Electrical'].fillna('NA', inplace=True)
data_combine['KitchenQual'].fillna('NA', inplace=True)
data_combine['Functional'].fillna('NA', inplace=True)
data_combine['SaleType'].fillna('NA', inplace=True)

data_combine['Alley'].fillna('NA', inplace=True)
data_combine['FireplaceQu'].fillna('NA', inplace=True)
data_combine['PoolQC'].fillna('NA', inplace=True)
data_combine['Fence'].fillna('NA', inplace=True)
data_combine['MiscFeature'].fillna('NA', inplace=True)

data_combine['LotFrontage'].fillna(data_combine['LotFrontage'].median(), inplace=True)
data_combine['LotFrontage'+'_NA'] = np.where(data_combine['LotFrontage'].isnull(), 1, 0)

data_combine['MasVnrType'].fillna('NA', inplace=True)
data_combine['MasVnrArea'].fillna(data_combine['LotFrontage'].median(), inplace=True)
data_combine['MasVnrArea'+'_NA'] = np.where(data_combine['LotFrontage'].isnull(), 1, 0)


In [190]:
quality = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'GarageQual', 'GarageCond', 'PoolQC' 
           ,'BsmtFinType1','BsmtFinType2', 'Fence', ]
for var in quality:
        print(var, ' values: ', data_combine[var].unique())

ExterQual  values:  ['Gd' 'TA' 'Ex' 'Fa']
ExterCond  values:  ['TA' 'Gd' 'Fa' 'Po' 'Ex']
BsmtQual  values:  ['Gd' 'TA' 'Ex' 'NA' 'Fa']
BsmtCond  values:  ['TA' 'Gd' 'NA' 'Fa' 'Po']
BsmtExposure  values:  ['No' 'Gd' 'Mn' 'Av' 'NA']
HeatingQC  values:  ['Ex' 'Gd' 'TA' 'Fa' 'Po']
KitchenQual  values:  ['Gd' 'TA' 'Ex' 'Fa' 'NA']
FireplaceQu  values:  ['NA' 'TA' 'Gd' 'Fa' 'Ex' 'Po']
GarageQual  values:  ['TA' 'Fa' 'Gd' 'NA' 'Ex' 'Po']
GarageCond  values:  ['TA' 'Fa' 'NA' 'Gd' 'Po' 'Ex']
PoolQC  values:  ['NA' 'Ex' 'Fa' 'Gd']
BsmtFinType1  values:  ['GLQ' 'ALQ' 'Unf' 'Rec' 'BLQ' 'NA' 'LwQ']
BsmtFinType2  values:  ['Unf' 'BLQ' 'NA' 'ALQ' 'Rec' 'LwQ' 'GLQ']
Fence  values:  ['NA' 'MnPrv' 'GdWo' 'GdPrv' 'MnWw']


In [0]:
# transform the quality-related categorical value to discrete numerical number
Quality_dict = {'Ex': 5 , 'Gd':4 , 'TA':3 , 'Fa':2 , 'Po':1 , 'None': 0 , 'NA':0 , 'Av':3, 'Mn':2, 'GLQ':6, 'ALQ':5, 'BLQ':4,'Rec':3,'LwQ':2,'Unf':1 , 
                'GdPrv':4	, 'MnPrv':3 , 'GdWo':2 , 'MnWw':1}

data_combine[quality]=data_combine[quality].replace(Quality_dict)  

In [192]:
# find categorical variables
categorical = [var for var in data_combine.columns if data_combine[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))
print (categorical)

There are 30 categorical variables
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'BsmtExposure', 'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageType', 'GarageFinish', 'PavedDrive', 'MiscFeature', 'SaleType', 'SaleCondition']


In [193]:
# find numerical variables
        
numerical = [var for var in data_combine.columns if data_combine[var].dtype!='O']
print('There are {} numerical variables'.format(len(numerical)))

There are 52 numerical variables


In [194]:
#print(train.shape,test.shape)

discrete = []
for var in numerical:
    if len(data_combine[var].unique())<25:
        print(var, ' values: ', data_combine[var].unique())
        discrete.append(var)
        
print('There are {} discrete variables'.format(len(discrete)))

MSSubClass  values:  [ 60  20  70  50 190  45  90 120  30  85  80 160  75 180  40 150]
OverallQual  values:  [ 7  6  8  5  9  4 10  3  1  2]
OverallCond  values:  [5 8 6 7 4 2 3 9 1]
ExterQual  values:  [4 3 5 2]
ExterCond  values:  [3 4 2 1 5]
BsmtQual  values:  [4 3 5 0 2]
BsmtCond  values:  [3 4 0 2 1]
BsmtFinType1  values:  [6 5 1 3 4 0 2]
BsmtFinType2  values:  [1 4 0 5 3 2 6]
HeatingQC  values:  [5 4 3 2 1]
BsmtFullBath  values:  [1. 0. 2. 3.]
BsmtHalfBath  values:  [0. 1. 2.]
FullBath  values:  [2 1 3 0 4]
HalfBath  values:  [1 0 2]
BedroomAbvGr  values:  [3 4 1 2 0 5 6 8]
KitchenAbvGr  values:  [1 2 3 0]
KitchenQual  values:  [4 3 5 2 0]
TotRmsAbvGrd  values:  [ 8  6  7  9  5 11  4 10 12  3  2 14 13 15]
Fireplaces  values:  [0 1 2 3 4]
FireplaceQu  values:  [0 3 4 2 5 1]
GarageCars  values:  [2. 3. 1. 0. 4. 5.]
GarageQual  values:  [3 2 4 0 5 1]
GarageCond  values:  [3 2 0 4 1 5]
PoolArea  values:  [  0 512 648 576 555 480 519 738 144 368 444 228 561 800]
PoolQC  values:  [0 5 

In [195]:
# find continous variable
continuous = [var for var in numerical if var not in discrete and var not in ['Id', 'SalePrice']]
continuous

['LotFrontage',
 'LotArea',
 'YearBuilt',
 'YearRemodAdd',
 'MasVnrArea',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'GarageYrBlt',
 'GarageArea',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',
 'MiscVal']

In [196]:
from scipy.stats import skew
import numpy as np
from sklearn.preprocessing import power_transform

#skewness = [var for var in continuous if abs(skew(train[var]))> 0.5]
#print("Before transform:")
#for var in skewness:
#  skewness=skew(train[var])
#  print(var,skewness)
#train[skewness]=power_transform(train[skewness], method='yeo-johnson')
#test[skewness]=power_transform(test[skewness], method='yeo-johnson')

#print("After transform:")
#for var in skewness:
#  skewness=skew(train[var])
#  print(skewness)

#for var in continuous:
#  skewness=skew(train[var])
#  skew_feature=[]
#  if abs(skew(train[var]))> 0.5:
#    skewness


for var in continuous:
  skewness=skew(data_combine[var])
  print(skewness)
  if abs(skewness) > 0.5:
    print(skew(data_combine[var]))
    print(var,"too large")
#    data_combine[var]=power_transform(data_combine[var], method='box-cox')
    data_combine[var]=np.log1p(data_combine[var]+1e-99)
    print(skew(data_combine[var]))
    print(var,"after transformed")


#for var in continuous:
#  skewness=skew(train[var])
#  print(skewness)
#  if abs(skewness) > 0.5:
#    print(skew(train[var]),skew(test[var]))
#    print(var,"too large")
##    train[var]=power_transform(train[var], method='box-cox')
#    train[var]=np.log1p(train[var]+1e-99)
#    test[var]=np.log1p(test[var]+1e-99)
#    print(skew(train[var]),skew(test[var]))
#    print(var,"after transformed")
    

1.674851564049772
1.674851564049772
LotFrontage too large
-0.9943302724007427
LotFrontage after transformed
12.822431401556724
12.822431401556724
LotArea too large
-0.50475055129892
LotArea after transformed
-0.5998055475020266
-0.5998055475020266
YearBuilt too large
-0.6262153888824539
YearBuilt after transformed
-0.4510203770788569
2.6148975452000953
2.6148975452000953
MasVnrArea too large
0.5069754621758876
MasVnrArea after transformed
1.4252301840932442
1.4252301840932442
BsmtFinSF1 too large
-0.6166318734180464
BsmtFinSF1 after transformed
4.146143269564016
4.146143269564016
BsmtFinSF2 too large
2.462483176985272
BsmtFinSF2 after transformed
0.9193392331173604
0.9193392331173604
BsmtUnfSF too large
-2.1552055052687025
BsmtUnfSF after transformed
1.1568941366699765
1.1568941366699765
TotalBsmtSF too large
-4.956293600642524
TotalBsmtSF after transformed
1.4696044169256821
1.4696044169256821
1stFlrSF too large
0.0648276815584268
1stFlrSF after transformed
0.8616747488436027
0.861674

In [197]:
# log transform the SalePrice
skewness=skew(y_target)
print(skewness)
if abs(skewness) > 0.5:
   y_target=np.log(y_target)
   print(y_target)

1.880940746034036
0       12.247694
1       12.109011
2       12.317167
3       11.849398
4       12.429216
5       11.870600
6       12.634603
7       12.206073
8       11.774520
9       11.678440
10      11.771436
11      12.751300
12      11.877569
13      12.540758
14      11.964001
15      11.790557
16      11.911702
17      11.407565
18      11.976659
19      11.842229
20      12.692503
21      11.845103
22      12.345835
23      11.774520
24      11.944708
25      12.454104
26      11.811547
27      12.631340
28      12.242887
29      11.134589
          ...    
1430    12.165980
1431    11.875831
1432    11.074421
1433    12.136187
1434    11.982929
1435    12.066811
1436    11.699405
1437    12.885671
1438    11.916389
1439    12.190959
1440    12.160029
1441    11.913713
1442    12.644328
1443    11.703546
1444    12.098487
1445    11.767568
1446    11.969717
1447    12.388394
1448    11.626254
1449    11.429544
1450    11.820410
1451    12.567551
1452    11.884489
1453    11

In [198]:

# To create the dummy feature via one-hot encoding

data_combine = pd.get_dummies(data_combine)
  
train = data_combine[:train.shape[0]]
test = data_combine[train.shape[0]:] 

print("size of train",train.shape)
print("size of train",test.shape)

#for var in categorical:
#      ordered_labels = train.groupby([var])['SalePrice'].mean().sort_values().index
#      ordinal_label = {k:i for i, k in enumerate(ordered_labels, 0)} 
#     if len(train[var].unique()) < len(test[var].unique()):
#         ordinal_label.update( {'Missing' : len(train[var].unique()) } )
#      train[var] = train[var].map(ordinal_label)
#      test[var] = test[var].map(ordinal_label)
#      test[var].fillna(len(ordinal_label), inplace=True)

size of train (1460, 257)
size of train (1459, 257)


In [199]:
# See the data again
train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingQC,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Fireplaces,FireplaceQu,GarageYrBlt,GarageCars,GarageArea,GarageQual,GarageCond,WoodDeckSF,...,Functional_Min2,Functional_Mod,Functional_NA,Functional_Sev,Functional_Typ,GarageType_2Types,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_NA,GarageFinish_Fin,GarageFinish_NA,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_N,PavedDrive_P,PavedDrive_Y,MiscFeature_Gar2,MiscFeature_NA,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_COD,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_NA,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,1,60,4.189655,9.042040,7,5,7.602900,2003,5.283204e+00,4,3,4,3,6,6.561031,1,1.000000e-99,5.017280,6.753438,5,6.753438,6.751101e+00,1.000000e-99,7.444833,1.0,0.0,2,1,3,1,4,8,0,0,2003.0,2.0,548.0,3,3,1.000000e-99,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2,20,4.394449,9.169623,6,8,7.589336,1976,1.000000e-99,3,3,4,3,5,6.886532,1,1.000000e-99,5.652489,7.141245,5,7.141245,1.000000e-99,1.000000e-99,7.141245,0.0,1.0,2,0,3,1,3,6,1,3,1976.0,2.0,460.0,3,3,5.700444e+00,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3,60,4.234107,9.328212,7,5,7.601902,2002,5.093750e+00,4,3,4,3,6,6.188264,1,1.000000e-99,6.075346,6.825460,5,6.825460,6.765039e+00,1.000000e-99,7.488294,1.0,0.0,2,1,3,1,4,6,1,3,2001.0,2.0,608.0,3,3,1.000000e-99,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,4,70,4.110874,9.164401,7,5,7.557995,1970,1.000000e-99,3,3,3,4,5,5.379897,1,1.000000e-99,6.293419,6.629363,4,6.869014,6.629363e+00,1.000000e-99,7.448916,1.0,0.0,1,0,3,1,4,7,1,4,1998.0,3.0,642.0,3,3,1.000000e-99,...,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
4,5,60,4.442651,9.565284,8,5,7.601402,2000,5.860786e+00,4,3,4,3,6,6.486161,1,1.000000e-99,6.196444,7.044033,5,7.044033,6.960348e+00,1.000000e-99,7.695758,1.0,0.0,2,1,4,1,4,9,1,3,2000.0,3.0,836.0,3,3,5.262690e+00,...,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0


In [0]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVC
import xgboost as xgb

In [0]:
X_train, X_test, y_train, y_test = train_test_split(train, y_target, test_size=0.2,
                                                    random_state=0)

In [0]:
training_vars = [var for var in X_train.columns if var not in ['Id', 'SalePrice']]
scaler = StandardScaler()
scaler.fit(X_train[training_vars])
X_train_sc = scaler.transform(X_train[training_vars])
X_test_sc = scaler.transform(X_test[training_vars])

Set up the training model

In [204]:
from sklearn.metrics import mean_squared_error
#n_estimators=1000
n_s = [200, 500 , 1000 , 1200 ,1500]
m_dep = [5 ,10 ,15, 20 ,25]
for n_es in n_s:
  for m_d in m_dep:
    forest = RandomForestRegressor(n_estimators=n_es , criterion='mse', max_depth=m_d ,random_state=1)
    forest.fit(X_train_sc, y_train)
    y_train_pred=forest.predict(X_train_sc)
    y_test_pred=forest.predict(X_test_sc)
    print( n_es, m_d,'MSE traion:' , mean_squared_error(y_train,y_train_pred),'test:',mean_squared_error(y_test,y_test_pred))
#scores = cross_val_score(estimator=forest, X=x_train_sc, y=y_train_sc, cv=10 , scoring= 'neg_mean_squared_error')
#print("Accuracy: %0.3f (+/- %0.3f) [%s]" % (scores.mean(), scores.std(), label) )

200 5 MSE traion: 0.01580416355354234 test: 0.022541386343780872
200 10 MSE traion: 0.003707932998346829 test: 0.018804580528824923
200 15 MSE traion: 0.00290931344888682 test: 0.018344301279509735
200 20 MSE traion: 0.0028714910274199065 test: 0.018426734447456416
200 25 MSE traion: 0.0028868484022226 test: 0.018672997524573427
500 5 MSE traion: 0.01571277077801813 test: 0.0224568429674686
500 10 MSE traion: 0.003613384985699784 test: 0.018661546459559065
500 15 MSE traion: 0.002809887981635261 test: 0.01832971284412599
500 20 MSE traion: 0.002783982006213972 test: 0.01832482400440096
500 25 MSE traion: 0.0027949226248564405 test: 0.018500336113120054
1000 5 MSE traion: 0.015690071838402656 test: 0.02241080812757509
1000 10 MSE traion: 0.0036384434727211876 test: 0.01873429212371327
1000 15 MSE traion: 0.002834082556389399 test: 0.018457861998710468
1000 20 MSE traion: 0.0028016373890675786 test: 0.018455130226203858
1000 25 MSE traion: 0.002811037463083298 test: 0.018531071776558564


In [0]:
from sklearn.decomposition import PCA
from sklearn.svm import SVR
from sklearn import metrics
def n_component_analysis(n,X_train, y_train, X_val, y_val, C=1.0, gamma='auto', kernel_type="linear"):
   
    pca = PCA(n_components=n)
    print("PCA begin with n_components: {}".format(n))
    pca.fit(X_train)

    X_train_pca = pca.transform(X_train)
    X_val_pca = pca.transform(X_val)

    print('SVR begin')
    clf1 = SVR(kernel=kernel_type,C=C,gamma=gamma)
    clf1.fit(X_train_pca, y_train)
    predictions=clf1.predict(X_val_pca)
    MSE = metrics.mean_squared_error(y_val,predictions)
    
    print("MSE: {}".format(MSE))
    return MSE

In [232]:
# Calling the PCA function above where kernel is linear by default and see if we ca improve our accuracy
n_s = np.linspace(0.8, 0.99, num=4)
C_s = [0.0001 , 0.001, 0.01 , 0.1 , 1.0 , 10 , 100]
gamma_s = [0.00001 , 0.0001 , 0.001, 0.01 , 0.1 , 1.0 , 10 , 100]
accuracy = []
n= 0.95
for c in C_s:
  for gamma in gamma_s:
    clf1 = SVR(kernel="rbf",C=c,gamma=gamma)
    clf1.fit(X_train_sc, y_train)
    predictions=clf1.predict(X_test_sc)
    MSE = metrics.mean_squared_error(y_test,predictions)
    
    print("gamma:",gamma, "C:", c ,"MSE:", MSE)
for n in n_s:
    tmp = n_component_analysis(n,X_train_sc, y_train, X_test_sc, y_test, C=1 , gamma=0.0001, kernel_type="rbf")
    accuracy.append(tmp)

gamma: 1e-05 C: 0.0001 MSE: 0.15220253473330925
gamma: 0.0001 C: 0.0001 MSE: 0.15204595849245006
gamma: 0.001 C: 0.0001 MSE: 0.15116033099413004
gamma: 0.01 C: 0.0001 MSE: 0.15150039518354738
gamma: 0.1 C: 0.0001 MSE: 0.15221061299816213
gamma: 1.0 C: 0.0001 MSE: 0.1522118666927457
gamma: 10 C: 0.0001 MSE: 0.15221186934201097
gamma: 100 C: 0.0001 MSE: 0.15221186934201097
gamma: 1e-05 C: 0.001 MSE: 0.1520358379886781
gamma: 0.0001 C: 0.001 MSE: 0.15047891002447245
gamma: 0.001 C: 0.001 MSE: 0.1416799287019063
gamma: 0.01 C: 0.001 MSE: 0.14481298723557556
gamma: 0.1 C: 0.001 MSE: 0.15220402210348075
gamma: 1.0 C: 0.001 MSE: 0.15222151973966877
gamma: 10 C: 0.001 MSE: 0.15222154623831613
gamma: 100 C: 0.001 MSE: 0.15222154623831613
gamma: 1e-05 C: 0.01 MSE: 0.15037556494945836
gamma: 0.0001 C: 0.01 MSE: 0.1356625955835791
gamma: 0.001 C: 0.01 MSE: 0.08178319287537124
gamma: 0.01 C: 0.01 MSE: 0.10968024053655166
gamma: 0.1 C: 0.01 MSE: 0.15221160346768448
gamma: 1.0 C: 0.01 MSE: 0.15246030

In [0]:
svr = SVR(kernel="rbf",C=1.0,gamma=0.0001)

In [234]:
from sklearn.ensemble import VotingRegressor
ensemble = [('RandomForest',forest),('SVR',svr)]
vr = VotingRegressor(ensemble)
vr.fit(X_train_sc, y_train)
#predictions=.predict(X_test_sc)
y_train_pred_vr=vr.predict(X_train_sc)
y_test_pred_vr=vr.predict(X_test_sc)
print('MSE traion:' , mean_squared_error(y_train,y_train_pred_vr),'test:',mean_squared_error(y_test,y_test_pred_vr))

MSE traion: 0.0066919907664050255 test: 0.018432562396575917


In [236]:
# XGBooster regression model:

for max_depth in range(3,25,3):
  for min_child_weight in range(1,8,2):
    xgb_model = xgb.XGBRegressor(max_depth = max_depth, min_child_weight = min_child_weight)
    xgb_model.fit(X_train_sc, y_train,eval_metric='rmse')
    y_train_pred_xgb=xgb_model.predict(X_train_sc)
    y_test_pred_xgb=xgb_model.predict(X_test_sc)
    print('max_depth=',max_depth,'min_child_weight',min_child_weight) 
    print('MSE traion:' , mean_squared_error(y_train,y_train_pred_xgb),'test:',mean_squared_error(y_test,y_test_pred_xgb))

xgb_model = xgb.XGBRegressor()
xgb_model.fit(X_train_sc, y_train,eval_metric='rmse')

y_train_pred_xgb=xgb_model.predict(X_train_sc)
y_test_pred_xgb=xgb_model.predict(X_test_sc)
print('MSE traion:' , mean_squared_error(y_train,y_train_pred_xgb),'test:',mean_squared_error(y_test,y_test_pred_xgb))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:34:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
max_depth= 3 min_child_weight 1
MSE traion: 0.00669250918943887 test: 0.016742131881376517
[02:34:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


max_depth= 3 min_child_weight 3
MSE traion: 0.007074710919373807 test: 0.01757490460650098
[02:34:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

max_depth= 24 min_child_weight 5
MSE traion: 0.0002291781134359674 test: 0.016971834452111045
[02:35:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


max_depth= 24 min_child_weight 7
MSE traion: 0.00045946507397761477 test: 0.01829164879913805
[02:35:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


MSE traion: 0.00669250918943887 test: 0.016742131881376517


In [237]:

xgb_model = xgb.XGBRegressor(max_depth = 18, min_child_weight = 1)
ensemble = [('RandomForest',forest),('XGBRegressor',xgb_model)]
vr = VotingRegressor(ensemble)
vr.fit(X_train_sc, y_train)
#predictions=.predict(X_test_sc)
y_train_pred_vr=vr.predict(X_train_sc)
y_test_pred_vr=vr.predict(X_test_sc)
print('MSE traion:' , mean_squared_error(y_train,y_train_pred_vr),'test:',mean_squared_error(y_test,y_test_pred_vr))

/usr/local/lib/python3.6/dist-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[02:36:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MSE traion: 0.0007512174168388347 test: 0.014799741869264848


In [238]:
final_model = vr
X_submission_sc = scaler.transform(test[training_vars])
print(X_submission_sc.shape)
print(X_train_sc.shape)
np.isnan(X_submission_sc).any()
#print(X_submission_sc[0:5,:])
y_submission_trans = final_model.predict(X_submission_sc)
y_submission = np.exp (y_submission_trans)
print (y_submission)
final = pd.DataFrame(list(zip(test['Id'], y_submission)), 
               columns =['Id', 'SalePrice']) 
final.head()

(1459, 256)
(1168, 256)
[120663.27767184 162266.27766484 184584.42818025 ... 156458.96901031
 113215.47374735 226259.87903638]


,Id,SalePrice
0,1461,120663.277672
1,1462,162266.277665
2,1463,184584.428180
3,1464,187743.509159
4,1465,188369.609974


In [0]:
final.to_csv("final_randomforest_XGB.csv",sep=',', index=False)

In [0]:
files.download("final_randomforest_XGB.csv")